# Bulwark

 Bulwark's goal is to let you check that your data meets your assumptions of what it should look like at any (and every) step in your code, without making you work too hard. For more detail about this project, please visit its [github repo](https://github.com/ZaxR/bulwark/tree/master).

Unlike other solution which works directly on data, Bulwark provide two ways to check data.
 1. Apply check function directly on dataframe
 2. Annotate a python function which transform data.

It provides some built-in tests and allows custom methods for tests. List of all built-in tests ("checks") can be found [here](https://bulwark.readthedocs.io/en/stable/bulwark.checks.html)

In [50]:
import bulwark.decorators as dc
import pandas as pd
import bulwark.checks as ck

# Solution 1 Apply check function directly on dataframe

This solution is pretty similar to other tools, you create a data frame, then you apply checks on that dataframe

In [51]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation",
               "relationship",
               "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
df_test = pd.read_csv("../data/adult_with_duplicates.csv", names=columns, header=1, sep=',', na_values=["null"])

df_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,-12.0,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,NaN,emp-by-pengfei,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,39.5,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
3,39.0,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
4,50.0,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


First we check the shape of dataframe, you can notice the actual shape is different from the expected shape

In [52]:
try:
    ck.is_shape(df_test, (32561, 15)).head()
    print("Test pass")
except Exception as e:
    print("**Exception**", e)

**Exception** Expected shape: (32561, 15)
		Actual shape:   (32606, 15)


In [55]:
# this time we check with a correct shape
try:
    ck.is_shape(df_test, (32606, 15)).head()
    print("Test pass")
except Exception as e:
    print("**Exception**", e)


Test pass


Second we check the value range of age column. Note by default the null value (e.g. NaN) is considered in range.

In [53]:
try:
    ck.has_vals_within_range(df_test, items={"age": (1,100)}).head()
    print("Test pass")
except Exception as e:
    print("**Exception**", e)

**Exception** ('Outside range', 0         True
1        False
2        False
3        False
4        False
         ...  
32601    False
32602    False
32603    False
32604    False
32605    False
Name: age, Length: 32606, dtype: bool)


In below test, we fill null value with 0, then we do the same test, you can notice we have one more row that is outside range.

In [54]:
try:
    df_test = df_test.fillna({"age": 0})
    df_test = df_test.astype({"age": int}, errors='ignore')
    ck.has_vals_within_range(df_test, items={"age": (1,100)}).head()
    print("Test pass")
except Exception as e:
    print("**Exception**", e)

**Exception** ('Outside range', 0         True
1         True
2        False
3        False
4        False
         ...  
32601    False
32602    False
32603    False
32604    False
32605    False
Name: age, Length: 32606, dtype: bool)


## Solution 2 Use function decorators

 We can apply checks/assertions on a python function by using the function decorators. But the function must return a pandas dataframe as the first return argument

Below function reads a csv file and returns a pandas df. We added three checks, if all checks pass, it returns the dataframe, otherwise it raises errors if any of the tests fail

In [40]:
# path of the valid data
valid_file_path = "../data/adult.csv"
test_file_path = "../data/adult_with_duplicates.csv"

In [41]:
# in version 1, we ignore the null value of the csv
@dc.HasNoNans()
@dc.IsShape((32561, 15))  # 10000 rows, 5 columns
@dc.HasValsWithinRange(items={"age": (1, 100)})  # min and max are inclusive here
def check_valid_csv_v1(file_path):
    columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation",
               "relationship",
               "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
    df = pd.read_csv(file_path, names=columns, header=None)
    return df

In [42]:
# Test passed with valid data
check_valid_csv_v1(valid_file_path)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [13]:
# in version 2, we specify the null value of the csv
@dc.HasNoNans()
@dc.IsShape((32561, 15))  # 10000 rows, 5 columns
@dc.HasValsWithinRange(items={"age": (1, 100)})  # min and max are inclusive here
def check_valid_csv_v2(file_path):
    columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation",
               "relationship",
               "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
    df = pd.read_csv(file_path, names=columns, header=None, sep=',\s+', na_values=["?"])
    return df


check_valid_csv_v2(valid_file_path)

<>:9: DeprecationWarning: invalid escape sequence \s
<>:9: DeprecationWarning: invalid escape sequence \s
/tmp/ipykernel_8009/3318884074.py:9: DeprecationWarning: invalid escape sequence \s
  df = pd.read_csv(file_path, names=columns, header=None, sep=',\s+', na_values=["?"])
/home/pliu/.cache/pypoetry/virtualenvs/dataqualityandvalidation-6LYnP9NJ-py3.8/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
/tmp/ipykernel_8009/3318884074.py:9: DeprecationWarning: invalid escape sequence \s
  df = pd.read_csv(file_path, names=columns, header=None, sep=',\s+', na_values=["?"])


AssertionError: ((27, 'workclass'), (61, 'workclass'), (69, 'workclass'), (77, 'workclass'), (106, 'workclass'), (128, 'workclass'), (149, 'workclass'), (154, 'workclass'), (160, 'workclass'), (187, 'workclass'), (201, 'workclass'), (221, 'workclass'), (226, 'workclass'), (243, 'workclass'), (266, 'workclass'), (297, 'workclass'), (312, 'workclass'), (326, 'workclass'), (346, 'workclass'), (347, 'workclass'), (354, 'workclass'), (397, 'workclass'), (408, 'workclass'), (430, 'workclass'), (431, 'workclass'), (449, 'workclass'), (459, 'workclass'), (471, 'workclass'), (484, 'workclass'), (486, 'workclass'), (499, 'workclass'), (511, 'workclass'), (515, 'workclass'), (517, 'workclass'), (518, 'workclass'), (539, 'workclass'), (576, 'workclass'), (580, 'workclass'), (590, 'workclass'), (591, 'workclass'), (596, 'workclass'), (648, 'workclass'), (656, 'workclass'), (668, 'workclass'), (671, 'workclass'), (686, 'workclass'), (695, 'workclass'), (734, 'workclass'), (788, 'workclass'), (789, '

You can notice, the test HasNoNans did not pass, and it returns all the cell (row number,colum name) that did not pass the test.

In below function, we will do some data cleaning, we replace null value by 0. and change the column type from float to int. So this time, the hasNoNans() check should pass. But the hasValsWithinRange will fail, because all null value will be 0, and we have introduced negative value in it

In [47]:
@dc.HasNoNans()
@dc.IsShape((32561, 15))  # 10000 rows, 5 columns
@dc.HasValsWithinRange(items={"age": (1, 100)})  # min and max are inclusive here
def check_valid_csv_v3(file_path):
    columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation",
               "relationship",
               "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
    df = pd.read_csv(file_path, names=columns, header=1, sep=',', na_values=["null"])
    df = df.fillna({"age": 0})
    df = df.astype({"age": int}, errors='ignore')
    return df


check_valid_csv_v3(test_file_path)

AssertionError: ('Outside range', 0         True
1         True
2        False
3        False
4        False
         ...  
32601    False
32602    False
32603    False
32604    False
32605    False
Name: age, Length: 32606, dtype: bool)

Note the output prints all the evaluation result of the column. True means the value is outside the given range, which cause the check to fail.